第5章: 係り受け解析
================

夏目漱石の小説『吾輩は猫である』の文章（neko.txt）をCaboChaを使って係り受け解析し，その結果をneko.txt.cabochaというファイルに保存せよ．このファイルを用いて，以下の問に対応するプログラムを実装せよ．




* download

https://drive.google.com/folderview?id=0B4y35FiV1wh7cGRCUUJHVTNJRnM&usp=sharing#list からダウンロード

* インストール

```bash
$ tar xvf cabocha-0.69.tar.bz2
$ cd cabocha-0.69/
$ # brew で入れたりソースから入れたり人それぞれなので which mecab-config
$ ./configure --with-mecab-config=`which mecab-config` --with-charset=utf8
$ make
$ sudo make install
$
$ cabocha
太郎はこの本を二郎を見た女性に渡した。
  太郎は-----------D
      この-D       |
        本を---D   |
        二郎を-D   |
            見た-D |
            女性に-D
            渡した。
EOS
^C
$
$ cd python
$ python setup.py install
$ 
```


pyenv, pyenv-virtualenv を使っていて
jupyter という virtualenvを作っている場合は

```
$ pyenv activate jupyter
$ python setup.py install
```

```
$ ptpython
>>> import CaboCha

>>> CaboCha.VERSION
'0.69'

>>> quit()
$
```


In [15]:
import CaboCha

_c = CaboCha.Parser()

with open("resources/neko.txt") as infile:
    with open("output/neko.txt.cabocha","w") as outfile:
        outfile.write(_c.parse(infile.read()).toString(CaboCha.FORMAT_LATTICE))
    
    

40. 係り受け解析結果の読み込み（形態素）
----------------------------------

形態素を表すクラスMorphを実装せよ．このクラスは表層形（surface），基本形（base），品詞（pos），品詞細分類1（pos1）をメンバ変数に持つこととする．さらに，CaboChaの解析結果（neko.txt.cabocha）を読み込み，各文をMorphオブジェクトのリストとして表現し，3文目の形態素列を表示せよ．



In [83]:
from prettytable import PrettyTable

class Morph():
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base   = base
        self.pos = pos
        self.pos1 = pos1


def make_list(filename):
    with open(filename) as f:
        lines = f.readlines()
        line_list = []
        lines_list = []
        for line in lines[:300]:
            temp_list = line.split() #space区切り
            for l in temp_list:
                line_list += l.split(",") #カンマ区切り(全部別れる)
            lines_list.append(line_list)
            line_list = []
    return lines_list

        
def make_sentenses_list(lines_list):
    sentences = [] # 文章全部
    sentence = [] # 一文章
    for line_list in lines_list:
        if line_list[0] == '*':
            pass
        elif line_list[2] == "句点" or line_list[0] == 'EOS':
            sentence.append(line_list)
            sentences.append(sentence)
            sentence = []
        else:
            sentence.append(line_list)
        
    return sentences
            
    
if __name__ == '__main__':
    lines_list = make_list('./output/neko.txt.cabocha')
    sentences = make_sentenses_list(lines_list)
    
    morph_sentence = []
    morph_sentences = []

    for sentence in sentences:
        for word in sentence: #１単語
            morph = Morph(
                surface=word[0], 
                base=word[7],
                pos=word[1],
                pos1=word[2])
            morph_sentence.append(morph)
        morph_sentences.append(morph_sentence)
        morph_sentence = []

    t = PrettyTable(['surface', 'base', 'pos', 'pos1'])
    for m in morph_sentences[3]:
        t.add_row([m.surface, m.base, m.pos, m.pos1])
    print(t)

        
        

+--------------+----------+--------+----------+
|   surface    |   base   |  pos   |   pos1   |
+--------------+----------+--------+----------+
|      何      |    何    |  名詞  |  代名詞  |
|     でも     |   でも   |  助詞  |  副助詞  |
|    薄暗い    |  薄暗い  | 形容詞 |   自立   |
|   じめじめ   | じめじめ |  副詞  |   一般   |
|      し      |   する   |  動詞  |   自立   |
|      た      |    た    | 助動詞 |    *     |
|      所      |    所    |  名詞  |  非自立  |
|      で      |    で    |  助詞  |  格助詞  |
| ニャーニャー |    *     |  名詞  |   一般   |
|     泣い     |   泣く   |  動詞  |   自立   |
|      て      |    て    |  助詞  | 接続助詞 |
|    いた事    |  いた事  |  名詞  |   一般   |
|     だけ     |   だけ   |  助詞  |  副助詞  |
|      は      |    は    |  助詞  |  係助詞  |
|     記憶     |   記憶   |  名詞  | サ変接続 |
|      し      |   する   |  動詞  |   自立   |
|      て      |    て    |  助詞  | 接続助詞 |
|     いる     |   いる   |  動詞  |  非自立  |
|      。      |    。    |  記号  |   句点   |
+--------------+----------+--------+----------+


41. 係り受け解析結果の読み込み（文節・係り受け）
----------------------------------
40に加えて，文節を表すクラスChunkを実装せよ．このクラスは形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つこととする．さらに，入力テキストのCaboChaの解析結果を読み込み，１文をChunkオブジェクトのリストとして表現し，8文目の文節の文字列と係り先を表示せよ．第5章の残りの問題では，ここで作ったプログラムを活用せよ．

42. 係り元と係り先の文節の表示
----------------------------------
係り元の文節と係り先の文節のテキストをタブ区切り形式ですべて抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

43. 名詞を含む文節が動詞を含む文節に係るものを抽出
----------------------------------
名詞を含む文節が，動詞を含む文節に係るとき，これらをタブ区切り形式で抽出せよ．ただし，句読点などの記号は出力しないようにせよ．

44. 係り受け木の可視化
----------------------------------
与えられた文の係り受け木を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

45. 動詞の格パターンの抽出
----------------------------------
今回用いている文章をコーパスと見なし，日本語の述語が取りうる格を調査したい． 動詞を述語，動詞に係っている文節の助詞を格と考え，述語と格をタブ区切り形式で出力せよ． ただし，出力は以下の仕様を満たすようにせよ．

動詞を含む文節において，最左の動詞の基本形を述語とする
述語に係る助詞を格とする
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める  で
見る    は を
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語と格パターンの組み合わせ
「する」「見る」「与える」という動詞の格パターン（コーパス中で出現頻度の高い順に並べよ）
46. 動詞の格フレーム情報の抽出
45のプログラムを改変し，述語と格パターンに続けて項（述語に係っている文節そのもの）をタブ区切り形式で出力せよ．45の仕様に加えて，以下の仕様を満たすようにせよ．

項は述語に係っている文節の単語列とする（末尾の助詞を取り除く必要はない）
述語に係る文節が複数あるときは，助詞と同一の基準・順序でスペース区切りで並べる
「吾輩はここで始めて人間というものを見た」という例文（neko.txt.cabochaの8文目）を考える． この文は「始める」と「見る」の２つの動詞を含み，「始める」に係る文節は「ここで」，「見る」に係る文節は「吾輩は」と「ものを」と解析された場合は，次のような出力になるはずである．

始める  で      ここで
見る    は を   吾輩は ものを

47. 機能動詞構文のマイニング
----------------------------------
動詞のヲ格にサ変接続名詞が入っている場合のみに着目したい．46のプログラムを以下の仕様を満たすように改変せよ．

「サ変接続名詞+を（助詞）」で構成される文節が動詞に係る場合のみを対象とする
述語は「サ変接続名詞+を+動詞の基本形」とし，文節中に複数の動詞があるときは，最左の動詞を用いる
述語に係る助詞（文節）が複数あるときは，すべての助詞をスペース区切りで辞書順に並べる
述語に係る文節が複数ある場合は，すべての項をスペース区切りで並べる（助詞の並び順と揃えよ）
例えば「別段くるにも及ばんさと、主人は手紙に返事をする。」という文から，以下の出力が得られるはずである．

返事をする      と に は        及ばんさと 手紙に 主人は
このプログラムの出力をファイルに保存し，以下の事項をUNIXコマンドを用いて確認せよ．

コーパス中で頻出する述語（サ変接続名詞+を+動詞）
コーパス中で頻出する述語と助詞パターン


48. 名詞から根へのパスの抽出
----------------------------------
文中のすべての名詞を含む文節に対し，その文節から構文木の根に至るパスを抽出せよ． ただし，構文木上のパスは以下の仕様を満たすものとする．

各文節は（表層形の）形態素列で表現する
パスの開始文節から終了文節に至るまで，各文節の表現を"->"で連結する
「吾輩はここで始めて人間というものを見た」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た


49. 名詞間の係り受けパスの抽出
----------------------------------
文中のすべての名詞句のペアを結ぶ最短係り受けパスを抽出せよ．ただし，名詞句ペアの文節番号がiiとjj（i<ji<j）のとき，係り受けパスは以下の仕様を満たすものとする．

問題48と同様に，パスは開始文節から終了文節に至るまでの各文節の表現（表層形の形態素列）を"->"で連結して表現する
文節iiとjjに含まれる名詞句はそれぞれ，XとYに置換する
また，係り受けパスの形状は，以下の2通りが考えられる．

文節iiから構文木の根に至る経路上に文節jjが存在する場合: 文節iiから文節jjのパスを表示
上記以外で，文節iiと文節jjから構文木の根に至る経路上で共通の文節kkで交わる場合: 文節iiから文節kkに至る直前のパスと文節jjから文節kkに至る直前までのパス，文節kkの内容を"|"で連結して表示
例えば，「吾輩はここで始めて人間というものを見た。」という文（neko.txt.cabochaの8文目）から，次のような出力が得られるはずである．

```
Xは | Yで -> 始めて -> 人間という -> ものを | 見た
Xは | Yという -> ものを | 見た
Xは | Yを | 見た
Xで -> 始めて -> Y
Xで -> 始めて -> 人間という -> Y
Xという -> Y
```